# JUICE RPWI HF SID3 (Full): L1a QL -- 2024/9/2

# Import lib

In [ ]:
import copy
import datetime
import glob
import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sys

from matplotlib import gridspec

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib  as juice_cal
import juice_cdf_lib  as juice_cdf
import juice_math_lib as juice_math
import juice_sid3_lib as juice_sid3
import juice_spec_lib as juice_spec

# Setting and Read CDF file: set by User

In [ ]:
# *** Dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump
# *** CAL ***
unit_mode = 2                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 0                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all
# *** Unit ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 0                           # [Time]      0: Epoch   1: data number
gap_mode  = 0                           # [gap]       0: n/a     1: gap
# *** Frenquency in Linear ***
f_mode_min = 80;  f_mode_max = 2068     # 80 - 2068 : as same as SID-4/20
# *** Power range set ***
p_raw_max = 9.5                         # background: 7.5   CAL: 10
p_raw_min = 2.0                         # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
N_ch0     = 4                           # [channel]   2: 2-ch    3: 3-ch                     >3: any
comp_mode = 4                           # [Complex]   0: Poweer  1: Matrix   3: Matrix-2D    >3: any

In [ ]:
E_min = '2024-08-20 22:10:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S")
E_max = '2024-08-20 23:10:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S")
t_min = 0;  t_max = 0

In [ ]:
# *** Flight data: Ver.2 ***
"""
# 20240126
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/01/26/'        # CDF data folder
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240126T083835_V01.cdf',
                 ]
# 20240701
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/07/01/'        # CDF data folder
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240701T011127_V01.cdf',
                 ]
# 20240706
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/07/06/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240706T121009_V01.cdf',
                  'JUICE_L1a_RPWI-HF-SID3_20240706T175202_V01.cdf',
                 ]
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/07/06/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID03_20240706T121009-20240706T131448_V01.cdf',
                  'JUICE_L1a_RPWI-HF-SID03_20240706T175202-20240706T183012_V01.cdf',
                 ]
# 202408
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/08/19/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240819T202636_V01.cdf']
"""
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/08/20/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240820T180734_V01.cdf']
"""
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/08/22/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240822T095914_V01.cdf']
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/08/23/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240823T035818_V01.cdf']
"""

In [ ]:
# *** Ground Test - Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'
data_name_list = [#'SID03_20231024-0036.cdf',                # 10/10/10mV, 0/90/0deg,  1.55MHz
                  #'SID03-2ch-comp0-20231117-1438.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                  #'SID03-2ch-comp1-20231117-1448.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                  #'SID03-2ch-comp2-20231117-1500.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                  #'SID03-3ch-comp0-20231117-1424.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                  'SID03-3ch-comp1-20231117-1418.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                  #'SID03-3ch-comp2-20231117-1429.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                  #'SID03-3ch-comp3-20231117-1432.cdf',      # 10/10/10mV, 90/0/0deg,  1.55MHz
                 ]
"""
# 202310 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202311_FS/cdf/'
data_name_list = ['SID03_Seq03.cdf',
                  'SID03_Seq04.cdf',
                  'SID03_Seq10.cdf',
                  'SID03_Seq11.cdf',
                  'SID03_Seq12.cdf',
                  'SID03_Seq13.cdf',
                  'SID03_Seq15.cdf',
                  'SID03_Seq17.cdf',
                  'SID03_Seq14.cdf',
                 ]
# 202401 -- ErrorCheck
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202401_FS/cdf/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_comp2.cdf',
                 ]
# 202405 -- dryrun check
# TEST
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/1_first_parts/'
# COMP1->2, 2->1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T040602-20000108T041954_V01___TMIDX_00000.bin.cdf',  # pc2_rpwi_hf_SID3_without_masking_2024_05_15_14_02_53
                 ]
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T042923-20000108T044315_V01___TMIDX_00000.bin.cdf',  # pc2_rpwi_hf_SID3_with_masking_2024_05_15_14_26_24
                 ]
# COMP2
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T050103-20000108T051303_V01___TMIDX_00000.bin.cdf',  # pc2_SC_roll_RPW-SEQ-12i-1-Stack_RPWI_2_2024_05_15_14_57_41/TM00
                  'JUICE_L1a_RPWI-HF-SID3_20000108T051318-20000108T051533_V01___TMIDX_00001.bin.cdf',  # pc2_SC_roll_RPW-SEQ-12i-1-Stack_RPWI_2_2024_05_15_14_57_41/TM01
                 ]
# COMP
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T051649-20000108T052219_V01___TMIDX_00000.bin.cdf',  # pc2_RPWI_combined-Stack_RPWI_3_2024_05_15_15_13_36
                 ]
#
# LGA: COMP-1
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/2_LGA-dryrun/'   
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T052829-20000108T053159_V01___TMIDX_00000.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-1_2024_05_15_15_25_20
                  'JUICE_L1a_RPWI-HF-SID3_20000108T054433-20000108T055503_V01___TMIDX_00000.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-2_2024_05_15_15_41_26/TM00
                  'JUICE_L1a_RPWI-HF-SID3_20000108T055533-20000108T060533_V01___TMIDX_00001.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-2_2024_05_15_15_41_26/TM01
                  'JUICE_L1a_RPWI-HF-SID3_20000108T060603-20000108T060703_V01___TMIDX_00002.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-2_2024_05_15_15_41_26/TM02
                  'JUICE_L1a_RPWI-HF-SID3_20000108T064409-20000108T064839_V01___TMIDX_00000.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13b_2024_05_15_16_40_53
                 ]
# EGA: COMP-1 & -2
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/3_EGA-dryrun/'   
# COMP-2
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T065600-20000108T065800_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04a_2024_05_15_16_52_23
                  'JUICE_L1a_RPWI-HF-SID3_20000108T065858-20000108T070058_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04b-1_2024_05_15_16_55_43
                  'JUICE_L1a_RPWI-HF-SID3_20000108T070151-20000108T070351_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04c_2024_05_15_16_58_39
                  'JUICE_L1a_RPWI-HF-SID3_20000108T070445-20000108T070645_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04b-2_2024_05_15_17_01_40
                  'JUICE_L1a_RPWI-HF-SID3_20000108T070750-20000108T070950_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04d_2024_05_15_17_04_35
                 ]
# COMP-1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T071058-20000108T071258_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12a-1_2024_05_15_17_07_46
                  'JUICE_L1a_RPWI-HF-SID3_20000108T071707-20000108T071907_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12b_2024_05_15_17_13_55
                  'JUICE_L1a_RPWI-HF-SID3_20000108T071951-20000108T072151_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12a-2_2024_05_15_17_16_42
                  'JUICE_L1a_RPWI-HF-SID3_20000108T072241-20000108T072441_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12c_2024_05_15_17_19_28
                  'JUICE_L1a_RPWI-HF-SID3_20000108T072530-20000108T072730_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12d_2024_05_15_17_22_18
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073057-20000108T073257_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12e_2024_05_15_17_27_45
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073349-20000108T073549_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12f_2024_05_15_17_30_37
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073635-20000108T073835_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12g_2024_05_15_17_33_23
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073920-20000108T074120_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12h_2024_05_15_17_36_08
                 ]
# COMP-2
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T074210-20000108T074410_V01___TMIDX_00000.bin.cdf',  # pc2_SC_roll_RPW-SEQ-12i-2_2024_05_15_17_38_55
                 ]
# COMP-1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T074511-20000108T074711_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12j_2024_05_15_17_41_59
                 ]

# 202406-7 -- dryrun check
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202406_EM3/cdf/'
# COMP1->2, 2->1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240614T161148-20240614T162539_V01___TMIDX_00000.bin.cdf',  
                 ]

# 202407 -- LEGA check
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202407_FS/cdf_20240716/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240716T152808-20240716T153933_V01.cdf',
                 ]
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202407_FS/cdf_20240718/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000101T011258-20000101T013701_V01___TMIDX_00000.bin.cdf',
                 ]
"""


In [ ]:
# *** Group read
"""
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202407_FS/cdf_20240716/'        # CDF data folder
data_name = '*SID3*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid3.hf_sid3_read(cdf)
    if i==0:
        data = data1;                                print(data.EuEu.shape)
    else:
        data = juice_sid3.hf_sid3_add(data, data1);  print(data.EuEu.shape)
data_name = os.path.split(data_name)[1];             print("data name:", data_name)

In [ ]:
data = juice_sid3.hf_sid3_shaping(data, cal_mode, N_ch0, comp_mode)

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
n_time0 = data.EuEu.shape[0];   n_freq0 = data.EuEu.shape[1]
f_min0 = data.frequency[0][0];  f_max0 = (max(np.ravel(data.frequency)))

print("date and time:", str_date)
print("    data size:", data.EuEu.shape, n_time0, n_freq0)
print("    Frequency:", f_min0,  "-", f_max0, "kHz")

In [ ]:
if N_ch0 > 3:
    N_ch0     = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
if comp_mode > 3:
    comp_mode = data.complex[0]
N_frequency = data.frequency.shape[1]

if (comp_mode == 0):
    if N_ch0 == 3:
        print("complex:", comp_mode, " (power 3-ch) <Table 6.2.2>")
    else:
        print("complex:", comp_mode, " (power 2-ch) <Table 6.2.3>")
    N_component = N_ch0
elif (comp_mode == 1):
    if N_ch0 == 3:
        print("complex:", comp_mode, " (nominal spectral matrix 3-ch) <Table 6.2.4>")
    else:
        print("complex:", comp_mode, " (nominal spectral matrix 2-ch) <Table 6.2.5>")
    N_component = N_ch0 * N_ch0
elif (comp_mode == 2):
    if N_ch0 == 3:
        N_component = 27 + 3
        print("complex:", comp_mode, " (Polarization separation 3-ch) <Table 6.2.6>")
    else:
        N_component = 12 + 3
        print("complex:", comp_mode, " (Polarization separation 2-ch) <Table 6.2.7>")
else:
    print("complex:", comp_mode, " (3D spectral matrix 3-ch) <Table 6.2.8>")
    N_component = 21

print("           Ch:", N_ch0, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("    data size:", data.EuEu.shape, n_time0, n_freq0)
print("  BG_downlink:", data.BG_downlink[0])
print("RFI_rejection:", data.RFI_rejection[0])
print("==> Component:", N_component, "   Num-frequency", N_frequency, "  Num-sweep:", n_time0)

# Spectrum Data

In [ ]:
T_HF  = data.T_HF_FPGA[n_time0//2];  T_RWI = (data.T_RWI_CH1[n_time0//2] + data.T_RWI_CH2[n_time0//2])/2
if T_HF  > 199 or T_HF  < -50:
   T_HF  = 25;  data.T_HF_FPGA[:] = math.nan
if T_RWI > 199 or T_RWI < -200:
   T_RWI = 25;  data.T_RWI_CH1[:] = math.nan;  data.T_RWI_CH2[:] = math.nan

print("Temperature: ", 'HF - {:.1f}'.format(T_HF), '  RWI1 - {:.1f}'.format(T_RWI))

In [ ]:
spec = copy.deepcopy(data)
spec = juice_cal.spec_cal(spec, 3,   unit_mode, band_mode, T_HF, T_RWI) # CAL
spec = juice_spec.hf_getspec_angle    (spec)                            # Coherency & ange
spec = juice_spec.hf_getspec_stokes   (spec)                            # Stokes and Polarization Parameters 
spec = juice_spec.hf_getspec_stokes_3d(spec)                            # 3D Stokes and Polarization Parameters

In [ ]:
p_max0 = p_raw_max + spec.cf/10;     p_min0 = p_raw_min + spec.cf/10
p_max = np.ceil(np.log10( np.amax( [np.amax(spec.EuEv_re), np.amax(spec.EvEw_re), np.amax(spec.EwEu_re), np.amax(spec.EuEv_im), np.amax(spec.EvEw_im), np.amax(spec.EwEu_im)] ) )*5)/5
p_min = p_min0

print("conversion factor:", '{:.1f}'.format(spec.cf), "    MAX-min:", '{:.1f}'.format(p_max), '({:.1f})'.format(p_max0), '{:.1f}'.format(p_min), '({:.1f})'.format(p_min0), "   str_unit:", spec.str)

In [ ]:
fig = plt.figure(figsize=(14, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)
else:
    ax1 = fig.add_subplot(4, 1, 1);  ax7 = fig.add_subplot(4, 1, 2);  ax8 = fig.add_subplot(4, 1, 3);  ax9 = fig.add_subplot(4, 1, 4)

if comp_mode != 2:
    ax1.plot(np.ravel(spec.EuEu),       '-r', linewidth=1.2, label='uu')
    ax1.plot(np.ravel(spec.EvEv),       '-g', linewidth=0.8, label='vv')
    ax1.plot(np.ravel(spec.EwEw),       '-b', linewidth=0.5, label='ww')
else: # Matrix
    ax1.plot(np.ravel(data.EuEu_NC),    '-r', linewidth=1.2, label='uu_NC')
    ax1.plot(np.ravel(data.EvEv_NC),    '-g', linewidth=0.8, label='vv_NC')
    ax1.plot(np.ravel(data.EwEw_NC),    '-b', linewidth=0.5, label='ww_NC')
    ax1.plot(np.ravel(data.EuEu_RC),    '.r', linewidth=1.2, label='uu_RC')
    ax1.plot(np.ravel(data.EvEv_RC),    '.g', linewidth=0.8, label='vv_RC')
    ax1.plot(np.ravel(data.EwEw_RC),    '.b', linewidth=0.5, label='ww_RC')
    ax1.plot(np.ravel(data.EuEu_LC),    '_r', linewidth=1.2, label='uu_LC')
    ax1.plot(np.ravel(data.EvEv_LC),    '_g', linewidth=0.8, label='vv_LC')
    ax1.plot(np.ravel(data.EwEw_LC),    '_b', linewidth=0.5, label='ww_LC')
ax1.plot(np.ravel(spec.BG_Eu),          ':r', linewidth=1.2, label='BG_uu')
ax1.plot(np.ravel(spec.BG_Ev),          ':g', linewidth=0.8, label='BG_vv')
ax1.plot(np.ravel(spec.BG_Ew),          ':b', linewidth=0.5, label='BG_ww')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width*10),  '-b', linewidth=1.0, label='width*10')
ax8.plot(np.ravel(data.T_HF_FPGA),      ':r', label='T (HK-FPGA)')
ax8.plot(np.ravel(data.T_RWI_CH1),      ':b', label='T (RWI1)')
ax8.plot(np.ravel(data.T_RWI_CH2),      ':g', label='T (RWI2)')
ax8.plot(np.ravel(data.cal_signal*10),      '-k', label='CAL')
ax8.plot(np.ravel(data.complex*10+4),       '_r', label='comp')
ax8.plot(np.ravel(data.RFI_rejection*10+2), '_b', label='RFI')
ax9.plot(np.ravel(data.epoch),              '.')
if comp_mode > 0:    # Matrix
    if comp_mode != 2:
        ax2.plot(np.ravel(spec.EuEv_re), '-r', linewidth=1.0, label='uv_re')
        ax2.plot(np.ravel(spec.EvEw_re), '-g', linewidth=0.8, label='vw_re')
        ax2.plot(np.ravel(spec.EwEu_re), '-b', linewidth=0.6, label='wu_re')
        ax2.plot(np.ravel(spec.EuEv_im), ':r', linewidth=1.2,  label='uv_im')
        ax2.plot(np.ravel(spec.EvEw_im), ':g', linewidth=1.0,  label='vw_im')
        ax2.plot(np.ravel(spec.EwEu_im), ':b', linewidth=0.8,  label='wu_im')
        ax3.plot(np.ravel(spec.E_DoPuv), '-r', linewidth=1.0, label='uv')
        ax3.plot(np.ravel(spec.E_DoPvw), '-g', linewidth=0.8, label='vw')
        ax3.plot(np.ravel(spec.E_DoPwu), '-b', linewidth=0.6, label='wu')
        ax4.plot(np.ravel(spec.E_DoLuv), '-r', linewidth=1.0, label='uv')
        ax4.plot(np.ravel(spec.E_DoLvw), '-g', linewidth=0.8, label='vw')
        ax4.plot(np.ravel(spec.E_DoLwu), '-b', linewidth=0.6, label='wu')
        ax5.plot(np.ravel(spec.E_DoCuv), '-r', linewidth=1.0, label='uv')
        ax5.plot(np.ravel(spec.E_DoCvw), '-g', linewidth=0.8, label='vw')
        ax5.plot(np.ravel(spec.E_DoCwu), '-b', linewidth=0.6, label='wu')
        ax6.plot(np.ravel(spec.E_ANGuv), '-r', linewidth=1.0, label='uv')
        ax6.plot(np.ravel(spec.E_ANGvw), '-g', linewidth=0.8, label='vw')
        ax6.plot(np.ravel(spec.E_ANGwu), '-b', linewidth=0.6, label='wu')
    else: # Matrix
        ax2.plot(np.ravel(spec.EuEv_re_NC), '-r', linewidth=1.0, label='uv_re NC')
        ax2.plot(np.ravel(spec.EvEw_re_NC), '-g', linewidth=0.8, label='vw_re NC')
        ax2.plot(np.ravel(spec.EwEu_re_NC), '-b', linewidth=0.6, label='wu_re NC')
        ax2.plot(np.ravel(spec.EuEv_im_NC), ':r', linewidth=1.2, label='uv_im NC')
        ax2.plot(np.ravel(spec.EvEw_im_NC), ':g', linewidth=1.0, label='vw_im NC')
        ax2.plot(np.ravel(spec.EwEu_im_NC), ':b', linewidth=0.8, label='wu_im NC')
        ax3.plot(np.ravel(spec.E_DoPuv_NC), '-r', linewidth=1.0, label='uv NC')
        ax3.plot(np.ravel(spec.E_DoPvw_NC), '-g', linewidth=0.8, label='vw NC')
        ax3.plot(np.ravel(spec.E_DoPwu_NC), '-b', linewidth=0.6, label='wu NC')
        ax4.plot(np.ravel(spec.E_DoLuv_NC), '-r', linewidth=1.0, label='uv NC')
        ax4.plot(np.ravel(spec.E_DoLvw_NC), '-g', linewidth=0.8, label='vw NC')
        ax4.plot(np.ravel(spec.E_DoLwu_NC), '-b', linewidth=0.6, label='wu NC')
        ax5.plot(np.ravel(spec.E_DoCuv_NC), '-r', linewidth=1.0, label='uv NC')
        ax5.plot(np.ravel(spec.E_DoCvw_NC), '-g', linewidth=0.8, label='vw NC')
        ax5.plot(np.ravel(spec.E_DoCwu_NC), '-b', linewidth=0.6, label='wu NC')
        ax6.plot(np.ravel(spec.E_ANGuv_NC), '-r', linewidth=1.0, label='uv NC')
        ax6.plot(np.ravel(spec.E_ANGvw_NC), '-g', linewidth=0.8, label='vw NC')
        ax6.plot(np.ravel(spec.E_ANGwu_NC), '-b', linewidth=0.6, label='wu NC')
        #
        ax2.plot(np.ravel(spec.EuEv_re_RC), '-r', linewidth=1.0, label='uv_re RC')
        ax2.plot(np.ravel(spec.EvEw_re_RC), '-g', linewidth=0.8, label='vw_re RC')
        ax2.plot(np.ravel(spec.EwEu_re_RC), '-b', linewidth=0.6, label='wu_re RC')
        ax2.plot(np.ravel(spec.EuEv_im_RC), ':r', linewidth=1.2, label='uv_im RC')
        ax2.plot(np.ravel(spec.EvEw_im_RC), ':g', linewidth=1.0, label='vw_im RC')
        ax2.plot(np.ravel(spec.EwEu_im_RC), ':b', linewidth=0.8, label='wu_im RC')
        ax3.plot(np.ravel(spec.E_DoPuv_RC), '-r', linewidth=1.0, label='uv RC')
        ax3.plot(np.ravel(spec.E_DoPvw_RC), '-g', linewidth=0.8, label='vw RC')
        ax3.plot(np.ravel(spec.E_DoPwu_RC), '-b', linewidth=0.6, label='wu RC')
        ax4.plot(np.ravel(spec.E_DoLuv_RC), '-r', linewidth=1.0, label='uv RC')
        ax4.plot(np.ravel(spec.E_DoLvw_RC), '-g', linewidth=0.8, label='vw RC')
        ax4.plot(np.ravel(spec.E_DoLwu_RC), '-b', linewidth=0.6, label='wu RC')
        ax5.plot(np.ravel(spec.E_DoCuv_RC), '-r', linewidth=1.0, label='uv RC')
        ax5.plot(np.ravel(spec.E_DoCvw_RC), '-g', linewidth=0.8, label='vw RC')
        ax5.plot(np.ravel(spec.E_DoCwu_RC), '-b', linewidth=0.6, label='wu RC')
        ax6.plot(np.ravel(spec.E_ANGuv_RC), '-r', linewidth=1.0, label='uv RC')
        ax6.plot(np.ravel(spec.E_ANGvw_RC), '-g', linewidth=0.8, label='vw RC')
        ax6.plot(np.ravel(spec.E_ANGwu_RC), '-b', linewidth=0.6, label='wu RC')
        #
        ax2.plot(np.ravel(spec.EuEv_re_LC), '-r', linewidth=1.0, label='uv_re LC')
        ax2.plot(np.ravel(spec.EvEw_re_LC), '-g', linewidth=0.8, label='vw_re LC')
        ax2.plot(np.ravel(spec.EwEu_re_LC), '-b', linewidth=0.6, label='wu_re LC')
        ax2.plot(np.ravel(spec.EuEv_im_LC), ':r', linewidth=1.2, label='uv_im LC')
        ax2.plot(np.ravel(spec.EvEw_im_LC), ':g', linewidth=1.0, label='vw_im LC')
        ax2.plot(np.ravel(spec.EwEu_im_LC), ':b', linewidth=0.8, label='wu_im LC')
        ax3.plot(np.ravel(spec.E_DoPuv_LC), '-r', linewidth=1.0, label='uv LC')
        ax3.plot(np.ravel(spec.E_DoPvw_LC), '-g', linewidth=0.8, label='vw LC')
        ax3.plot(np.ravel(spec.E_DoPwu_LC), '-b', linewidth=0.6, label='wu LC')
        ax4.plot(np.ravel(spec.E_DoLuv_LC), '-r', linewidth=1.0, label='uv LC')
        ax4.plot(np.ravel(spec.E_DoLvw_LC), '-g', linewidth=0.8, label='vw LC')
        ax4.plot(np.ravel(spec.E_DoLwu_LC), '-b', linewidth=0.6, label='wu LC')
        ax5.plot(np.ravel(spec.E_DoCuv_LC), '-r', linewidth=1.0, label='uv LC')
        ax5.plot(np.ravel(spec.E_DoCvw_LC), '-g', linewidth=0.8, label='vw LC')
        ax5.plot(np.ravel(spec.E_DoCwu_LC), '-b', linewidth=0.6, label='wu LC')
        ax6.plot(np.ravel(spec.E_ANGuv_LC), '-r', linewidth=1.0, label='uv LC')
        ax6.plot(np.ravel(spec.E_ANGvw_LC), '-g', linewidth=0.8, label='vw LC')
        ax6.plot(np.ravel(spec.E_ANGwu_LC), '-b', linewidth=0.6, label='wu LC')

ax1.set_ylabel(spec.str)
if comp_mode > 0:   # Matrix
    ax2.set_ylabel('Real & Img');     ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.')
    ax5.set_ylabel('Deg. Cir Pol.');  ax6.set_ylabel('Lin Deg.'); 
ax7.set_ylabel('frequency [kHz]');    ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');  ax9.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);     ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
if comp_mode > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.EuEu)) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
# ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
n_time = 4
fig = plt.figure(figsize=(14, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)
else:
    ax1 = fig.add_subplot(4, 1, 1);  ax7 = fig.add_subplot(4, 1, 2);  ax8 = fig.add_subplot(4, 1, 3);  ax9 = fig.add_subplot(4, 1, 4)

if comp_mode != 2:
    ax1.plot(np.ravel(spec.EuEu[n_time]),       '-r', linewidth=1.2, label='uu')
    ax1.plot(np.ravel(spec.EvEv[n_time]),       '-g', linewidth=0.8, label='vv')
    ax1.plot(np.ravel(spec.EwEw[n_time]),       '-b', linewidth=0.5, label='ww')
else: # Matrix
    ax1.plot(np.ravel(data.EuEu_NC[n_time]),    '-r', linewidth=1.2, label='uu_NC')
    ax1.plot(np.ravel(data.EvEv_NC[n_time]),    '-g', linewidth=0.8, label='vv_NC')
    ax1.plot(np.ravel(data.EwEw_NC[n_time]),    '-b', linewidth=0.5, label='ww_NC')
    ax1.plot(np.ravel(data.EuEu_RC[n_time]),    '.r', linewidth=1.2, label='uu_RC')
    ax1.plot(np.ravel(data.EvEv_RC[n_time]),    '.g', linewidth=0.8, label='vv_RC')
    ax1.plot(np.ravel(data.EwEw_RC[n_time]),    '.b', linewidth=0.5, label='ww_RC')
    ax1.plot(np.ravel(data.EuEu_LC[n_time]),    '_r', linewidth=1.2, label='uu_LC')
    ax1.plot(np.ravel(data.EvEv_LC[n_time]),    '_g', linewidth=0.8, label='vv_LC')
    ax1.plot(np.ravel(data.EwEw_LC[n_time]),    '_b', linewidth=0.5, label='ww_LC')
ax1.plot(np.ravel(spec.BG_Eu[n_time]),          ':r', linewidth=1.2, label='BG_uu')
ax1.plot(np.ravel(spec.BG_Ev[n_time]),          ':g', linewidth=0.8, label='BG_vv')
ax1.plot(np.ravel(spec.BG_Ew[n_time]),          ':b', linewidth=0.5, label='BG_ww')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency[n_time]),      '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step[n_time]*10),   '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width[n_time]*10),  '-b', linewidth=1.0, label='width*10')
ax8.plot(np.ravel(data.T_HF_FPGA[n_time]),      ':r', label='T (HK-FPGA)')
ax8.plot(np.ravel(data.T_RWI_CH1[n_time]),      ':b', label='T (RWI1)')
ax8.plot(np.ravel(data.T_RWI_CH2[n_time]),      ':g', label='T (RWI2)')
ax8.plot(np.ravel(data.cal_signal[n_time]*10),      '-k', label='CAL')
ax8.plot(np.ravel(data.complex[n_time]*10+4),       '_r', label='comp')
ax8.plot(np.ravel(data.RFI_rejection[n_time]*10+2), '_b', label='RFI')
ax9.plot(np.ravel(data.epoch[n_time]),              '.')
if comp_mode > 0:    # Matrix
    if comp_mode != 2:
        ax2.plot(np.ravel(spec.EuEv_re[n_time]), '-r', linewidth=1.0, label='uv_re')
        ax2.plot(np.ravel(spec.EvEw_re[n_time]), '-g', linewidth=0.8, label='vw_re')
        ax2.plot(np.ravel(spec.EwEu_re[n_time]), '-b', linewidth=0.6, label='wu_re')
        ax2.plot(np.ravel(spec.EuEv_im[n_time]), ':r', linewidth=1.2,  label='uv_im')
        ax2.plot(np.ravel(spec.EvEw_im[n_time]), ':g', linewidth=1.0,  label='vw_im')
        ax2.plot(np.ravel(spec.EwEu_im[n_time]), ':b', linewidth=0.8,  label='wu_im')
        ax3.plot(np.ravel(spec.E_DoPuv[n_time]), '-r', linewidth=1.0, label='uv')
        ax3.plot(np.ravel(spec.E_DoPvw[n_time]), '-g', linewidth=0.8, label='vw')
        ax3.plot(np.ravel(spec.E_DoPwu[n_time]), '-b', linewidth=0.6, label='wu')
        ax4.plot(np.ravel(spec.E_DoLuv[n_time]), '-r', linewidth=1.0, label='uv')
        ax4.plot(np.ravel(spec.E_DoLvw[n_time]), '-g', linewidth=0.8, label='vw')
        ax4.plot(np.ravel(spec.E_DoLwu[n_time]), '-b', linewidth=0.6, label='wu')
        ax5.plot(np.ravel(spec.E_DoCuv[n_time]), '-r', linewidth=1.0, label='uv')
        ax5.plot(np.ravel(spec.E_DoCvw[n_time]), '-g', linewidth=0.8, label='vw')
        ax5.plot(np.ravel(spec.E_DoCwu[n_time]), '-b', linewidth=0.6, label='wu')
        ax6.plot(np.ravel(spec.E_ANGuv[n_time]), '-r', linewidth=1.0, label='uv')
        ax6.plot(np.ravel(spec.E_ANGvw[n_time]), '-g', linewidth=0.8, label='vw')
        ax6.plot(np.ravel(spec.E_ANGwu[n_time]), '-b', linewidth=0.6, label='wu')
    else: # Matrix
        ax2.plot(np.ravel(spec.EuEv_re_NC[n_time]), '-r', linewidth=1.0, label='uv_re NC')
        ax2.plot(np.ravel(spec.EvEw_re_NC[n_time]), '-g', linewidth=0.8, label='vw_re NC')
        ax2.plot(np.ravel(spec.EwEu_re_NC[n_time]), '-b', linewidth=0.6, label='wu_re NC')
        ax2.plot(np.ravel(spec.EuEv_im_NC[n_time]), ':r', linewidth=1.2, label='uv_im NC')
        ax2.plot(np.ravel(spec.EvEw_im_NC[n_time]), ':g', linewidth=1.0, label='vw_im NC')
        ax2.plot(np.ravel(spec.EwEu_im_NC[n_time]), ':b', linewidth=0.8, label='wu_im NC')
        ax3.plot(np.ravel(spec.E_DoPuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
        ax3.plot(np.ravel(spec.E_DoPvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
        ax3.plot(np.ravel(spec.E_DoPwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
        ax4.plot(np.ravel(spec.E_DoLuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
        ax4.plot(np.ravel(spec.E_DoLvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
        ax4.plot(np.ravel(spec.E_DoLwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
        ax5.plot(np.ravel(spec.E_DoCuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
        ax5.plot(np.ravel(spec.E_DoCvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
        ax5.plot(np.ravel(spec.E_DoCwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
        ax6.plot(np.ravel(spec.E_ANGuv_NC[n_time]), '-r', linewidth=1.0, label='uv NC')
        ax6.plot(np.ravel(spec.E_ANGvw_NC[n_time]), '-g', linewidth=0.8, label='vw NC')
        ax6.plot(np.ravel(spec.E_ANGwu_NC[n_time]), '-b', linewidth=0.6, label='wu NC')
        #
        ax2.plot(np.ravel(spec.EuEv_re_RC[n_time]), '-r', linewidth=1.0, label='uv_re RC')
        ax2.plot(np.ravel(spec.EvEw_re_RC[n_time]), '-g', linewidth=0.8, label='vw_re RC')
        ax2.plot(np.ravel(spec.EwEu_re_RC[n_time]), '-b', linewidth=0.6, label='wu_re RC')
        ax2.plot(np.ravel(spec.EuEv_im_RC[n_time]), ':r', linewidth=1.2, label='uv_im RC')
        ax2.plot(np.ravel(spec.EvEw_im_RC[n_time]), ':g', linewidth=1.0, label='vw_im RC')
        ax2.plot(np.ravel(spec.EwEu_im_RC[n_time]), ':b', linewidth=0.8, label='wu_im RC')
        ax3.plot(np.ravel(spec.E_DoPuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
        ax3.plot(np.ravel(spec.E_DoPvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
        ax3.plot(np.ravel(spec.E_DoPwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
        ax4.plot(np.ravel(spec.E_DoLuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
        ax4.plot(np.ravel(spec.E_DoLvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
        ax4.plot(np.ravel(spec.E_DoLwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
        ax5.plot(np.ravel(spec.E_DoCuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
        ax5.plot(np.ravel(spec.E_DoCvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
        ax5.plot(np.ravel(spec.E_DoCwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
        ax6.plot(np.ravel(spec.E_ANGuv_RC[n_time]), '-r', linewidth=1.0, label='uv RC')
        ax6.plot(np.ravel(spec.E_ANGvw_RC[n_time]), '-g', linewidth=0.8, label='vw RC')
        ax6.plot(np.ravel(spec.E_ANGwu_RC[n_time]), '-b', linewidth=0.6, label='wu RC')
        #
        ax2.plot(np.ravel(spec.EuEv_re_LC[n_time]), '-r', linewidth=1.0, label='uv_re LC')
        ax2.plot(np.ravel(spec.EvEw_re_LC[n_time]), '-g', linewidth=0.8, label='vw_re LC')
        ax2.plot(np.ravel(spec.EwEu_re_LC[n_time]), '-b', linewidth=0.6, label='wu_re LC')
        ax2.plot(np.ravel(spec.EuEv_im_LC[n_time]), ':r', linewidth=1.2, label='uv_im LC')
        ax2.plot(np.ravel(spec.EvEw_im_LC[n_time]), ':g', linewidth=1.0, label='vw_im LC')
        ax2.plot(np.ravel(spec.EwEu_im_LC[n_time]), ':b', linewidth=0.8, label='wu_im LC')
        ax3.plot(np.ravel(spec.E_DoPuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
        ax3.plot(np.ravel(spec.E_DoPvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
        ax3.plot(np.ravel(spec.E_DoPwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')
        ax4.plot(np.ravel(spec.E_DoLuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
        ax4.plot(np.ravel(spec.E_DoLvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
        ax4.plot(np.ravel(spec.E_DoLwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')
        ax5.plot(np.ravel(spec.E_DoCuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
        ax5.plot(np.ravel(spec.E_DoCvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
        ax5.plot(np.ravel(spec.E_DoCwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')
        ax6.plot(np.ravel(spec.E_ANGuv_LC[n_time]), '-r', linewidth=1.0, label='uv LC')
        ax6.plot(np.ravel(spec.E_ANGvw_LC[n_time]), '-g', linewidth=0.8, label='vw LC')
        ax6.plot(np.ravel(spec.E_ANGwu_LC[n_time]), '-b', linewidth=0.6, label='wu LC')

ax1.set_ylabel(spec.str)
if comp_mode > 0:    # Matrix
    ax2.set_ylabel('Real & Img');     ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.')
    ax5.set_ylabel('Deg. Cir Pol.');  ax6.set_ylabel('Lin Deg.'); 
ax7.set_ylabel('frequency [kHz]');    ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');  ax9.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);     ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
if comp_mode > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(data.EuEu[n_time]) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
# ylim=[10**p_raw_min, 10**p_raw_max];  ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
print("Complex:", data.complex)
print("CAL_sig:", data.cal_signal)
print("CAL_onb:", data.onboard_cal)
print("POL_SW :", data.Pol_sep_SW)
print("POL_thr:", data.Pol_sep_thres)

### NaN

In [ ]:
if gap_mode == 1:
    for i in range(n_time0-1):
        if data.scet[i+1]-data.scet[i] > 60:
            print(i, data.epoch[i], i+1, data.epoch[i+1])
            spec.EuEu[i]    = math.nan;  spec.EvEv[i]    = math.nan;  spec.EwEw[i]    = math.nan;  spec.EuEu[i+1]    = math.nan;  spec.EvEv[i+1]    = math.nan;  spec.EwEw[i+1]    = math.nan  
            spec.E_DoLuv[i] = math.nan;  spec.E_DoLvw[i] = math.nan;  spec.E_DoLwu[i] = math.nan;  spec.E_DoLuv[i+1] = math.nan;  spec.E_DoLvw[i+1] = math.nan;  spec.E_DoLwu[i+1] = math.nan
            spec.E_DoCuv[i] = math.nan;  spec.E_DoCvw[i] = math.nan;  spec.E_DoCwu[i] = math.nan;  spec.E_DoCuv[i+1] = math.nan;  spec.E_DoCvw[i+1] = math.nan;  spec.E_DoCwu[i+1] = math.nan           
            spec.E_ANGuv[i] = math.nan;  spec.E_ANGvw[i] = math.nan;  spec.E_ANGwu[i] = math.nan;  spec.E_ANGuv[i+1] = math.nan;  spec.E_ANGvw[i+1] = math.nan;  spec.E_ANGwu[i+1] = math.nan

In [ ]:
Epoch_1d       = data.epoch.tolist()
freq_1d        = data.frequency[0].tolist()
freq1          = freq_1d[0:N_frequency]
freq_width_1d  = data.freq_width[0].tolist()
freq_width1    = freq_width_1d[0:N_frequency]

In [ ]:
print("N_samp:", data.N_samp[0], "   decimation:", juice_cdf._sample_rate(data.decimation[0]))
peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.argmax(peak_E)
print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.argmax(peak_E)
print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.argmax(peak_E)
print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))

In [ ]:
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
print(f"[First peak - {n_sweep1}]")
peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.argmax(peak_E)
print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.argmax(peak_E)
print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.argmax(peak_E)
print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Mid   peak - {n_sweep2}]")
peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.argmax(peak_E)
print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.argmax(peak_E)
print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.argmax(peak_E)
print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Last  peak - {n_sweep3}]")
peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.argmax(peak_E)
print("EuEu:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.argmax(peak_E)
print("EvEv:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.argmax(peak_E)
print("EwEw:", '{:.2e} V2 '.format(peak_E[peak_f]), '{:.2e} V '.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))

In [ ]:
n_sweep = n_time0-1;  peak_E = np.ravel(spec.EuEu[n_sweep]); peak_f = np.argmax(peak_E); 
print(f"[SWEEP - {n_sweep}]  Peak at", '{:.1f}'.format(freq_1d[peak_f]), "kHz", peak_f)
print("      EuEu:", '{:+.2e}'.format(np.ravel(spec.EuEu   [n_sweep])[peak_f]), spec.str, "     EvEv:", '{:+.2e}'.format(np.ravel(spec.EvEv   [n_sweep])[peak_f]), spec.str, "   EwEw:", '{:+.2e}'.format(np.ravel(spec.EwEw   [n_sweep])[peak_f]), spec.str)
print("   EuEv_re:", '{:+.2e}'.format(np.ravel(spec.EuEv_re[n_sweep])[peak_f]), spec.str, "  EuEv_im:", '{:+.2e}'.format(np.ravel(spec.EuEv_im[n_sweep])[peak_f]), spec.str, "  PHAuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAuv[n_sweep])[peak_f]), "  COHuv:", '{:+4.2f}'.format(np.ravel(spec.E_COHuv[n_sweep])[peak_f]), "  DoPuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoPuv[n_sweep])[peak_f]), "  DoLuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoLuv[n_sweep])[peak_f]), "  DoCuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoCuv[n_sweep])[peak_f]), "  ANGuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGuv[n_sweep])[peak_f]))
print("   EvEw_re:", '{:+.2e}'.format(np.ravel(spec.EvEw_re[n_sweep])[peak_f]), spec.str, "  EvEw_im:", '{:+.2e}'.format(np.ravel(spec.EvEw_im[n_sweep])[peak_f]), spec.str, "  PHAvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAvw[n_sweep])[peak_f]), "  COHvw:", '{:+4.2f}'.format(np.ravel(spec.E_COHvw[n_sweep])[peak_f]), "  DoPvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoPvw[n_sweep])[peak_f]), "  DoLvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoLvw[n_sweep])[peak_f]), "  DoCvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoCvw[n_sweep])[peak_f]), "  ANGvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGvw[n_sweep])[peak_f]))
print("   EwEu_re:", '{:+.2e}'.format(np.ravel(spec.EwEu_re[n_sweep])[peak_f]), spec.str, "  EwEu_im:", '{:+.2e}'.format(np.ravel(spec.EwEu_im[n_sweep])[peak_f]), spec.str, "  PHAwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAwu[n_sweep])[peak_f]), "  COHqu:", '{:+4.2f}'.format(np.ravel(spec.E_COHwu[n_sweep])[peak_f]), "  DoPwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoPwu[n_sweep])[peak_f]), "  DoLwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoLwu[n_sweep])[peak_f]), "  DoCwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoCwu[n_sweep])[peak_f]), "  ANGwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGwu[n_sweep])[peak_f]))
print('Stokes-UV: {:+.2e}'.format(np.ravel(spec.E_Iuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Quv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vuv[n_sweep])[peak_f]))
print('Stokes-VW: {:+.2e}'.format(np.ravel(spec.E_Ivw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vvw[n_sweep])[peak_f]))
print('Stokes-WU: {:+.2e}'.format(np.ravel(spec.E_Iwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vwu[n_sweep])[peak_f]))

In [ ]:
# [DEBUG] Complex-2 & -3: line & background intensity checks
peak_f = 15;  print (peak_f)  #55
for i in range(n_time0):
    print(i, data.complex[i],      '{:+.2e}'.format(spec.EuEu[i][peak_f]),          '{:+.2e}'.format(spec.EuEu_NC[i][peak_f]), data.num_NC[i][peak_f], '{:+.2e}'.format(data.EuEu_RC[i][peak_f]), data.num_RC[i][peak_f], '{:+.2e}'.format(data.EuEu_LC[i][peak_f]), data.num_LC[i][peak_f], '{:+.2e}'.format(data.EuiEui[i][peak_f]), '{:+.2e}'.format(data.EuqEuq[i][peak_f]))
    print(i, data.complex[i], '[UV] {:+.1e}'.format(spec.EuEu[i][peak_f]),    ' DoP:{:+3.2f}'.format(spec.E_DoPuv[i][peak_f]),    ' DoL:{:+3.2f}'.format(spec.E_DoLuv[i][peak_f]),    ' DoC:{:+3.2f}'.format(spec.E_DoCuv[i][peak_f]),    ' Ang:{:+3.0f}'.format(spec.E_ANGuv[i][peak_f]))
    print(i, data.complex[i], '[VW] {:+.1e}'.format(spec.EvEv[i][peak_f]),    ' DoP:{:+3.2f}'.format(spec.E_DoPvw[i][peak_f]),    ' DoL:{:+3.2f}'.format(spec.E_DoLvw[i][peak_f]),    ' DoC:{:+3.2f}'.format(spec.E_DoCvw[i][peak_f]),    ' Ang:{:+3.0f}'.format(spec.E_ANGvw[i][peak_f]))
    print(i, data.complex[i], '[WU] {:+.1e}'.format(spec.EwEw[i][peak_f]),    ' DoP:{:+3.2f}'.format(spec.E_DoPwu[i][peak_f]),    ' DoL:{:+3.2f}'.format(spec.E_DoLwu[i][peak_f]),    ' DoC:{:+3.2f}'.format(spec.E_DoCwu[i][peak_f]),    ' Ang:{:+3.0f}'.format(spec.E_ANGwu[i][peak_f]))
    # print(i, data.complex[i],  '[l] {:+.2e}'.format(spec.EuEu_NC[i][peak_f]),   'DoP:{:+.2e}'.format(spec.E_DoPuv_NC[i][peak_f]),   'DoL:{:+.2e}'.format(spec.E_DoLuv_NC[i][peak_f]),   'DoC:{:+.2e}'.format(spec.E_DoCuv_NC[i][peak_f]),   'ang:{:+.2e}'.format(spec.E_ANGuv_NC[i][peak_f]))
    # print(i, data.complex[i],  '[R] {:+.2e}'.format(spec.EuEu_RC[i][peak_f]),   'DoP:{:+.2e}'.format(spec.E_DoPuv_RC[i][peak_f]),   'DoL:{:+.2e}'.format(spec.E_DoLuv_RC[i][peak_f]),   'DoC:{:+.2e}'.format(spec.E_DoCuv_RC[i][peak_f]),   'ang:{:+.2e}'.format(spec.E_ANGuv_RC[i][peak_f]))
    # print(i, data.complex[i],  '[L] {:+.2e}'.format(spec.EuEu_LC[i][peak_f]),   'DoP:{:+.2e}'.format(spec.E_DoPuv_LC[i][peak_f]),   'DoL:{:+.2e}'.format(spec.E_DoLuv_LC[i][peak_f]),   'DoC:{:+.2e}'.format(spec.E_DoCuv_LC[i][peak_f]),   'ang:{:+.2e}'.format(spec.E_ANGuv_LC[i][peak_f]))

In [ ]:
spec.EE    = spec.EuEu + spec.EvEv + spec.EwEw
EuEu_2d    = spec.EuEu.transpose();     EuEu_med    = np.nanmedian(spec.EuEu,    axis=0)
EvEv_2d    = spec.EvEv.transpose();     EvEv_med    = np.nanmedian(spec.EvEv,    axis=0)
EwEw_2d    = spec.EwEw.transpose();     EwEw_med    = np.nanmedian(spec.EwEw,    axis=0)
EE_2d      = spec.EE.transpose();       EE_med      = np.nanmedian(spec.EuEu,    axis=0)
E_DoPuv_2d = spec.E_DoLuv.transpose();  E_DoPuv_med = np.nanmedian(spec.E_DoPuv, axis=0)
E_DoPvw_2d = spec.E_DoLvw.transpose();  E_DoPvw_med = np.nanmedian(spec.E_DoPvw, axis=0)
E_DoPwu_2d = spec.E_DoLwu.transpose();  E_DoPwu_med = np.nanmedian(spec.E_DoPwu, axis=0)
E_DoLuv_2d = spec.E_DoLuv.transpose();  E_DoLuv_med = np.nanmedian(spec.E_DoLuv, axis=0)
E_DoLvw_2d = spec.E_DoLvw.transpose();  E_DoLvw_med = np.nanmedian(spec.E_DoLvw, axis=0)
E_DoLwu_2d = spec.E_DoLwu.transpose();  E_DoLwu_med = np.nanmedian(spec.E_DoLwu, axis=0)
E_DoCuv_2d = spec.E_DoCuv.transpose();  E_DoCuv_med = np.nanmedian(spec.E_DoCuv, axis=0)
E_DoCvw_2d = spec.E_DoCvw.transpose();  E_DoCvw_med = np.nanmedian(spec.E_DoCvw, axis=0)
E_DoCwu_2d = spec.E_DoCwu.transpose();  E_DoCwu_med = np.nanmedian(spec.E_DoCwu, axis=0)
E_ANGuv_2d = spec.E_ANGuv.transpose();  E_ANGuv_med = np.nanmedian(spec.E_ANGuv, axis=0)
E_ANGvw_2d = spec.E_ANGvw.transpose();  E_ANGvw_med = np.nanmedian(spec.E_ANGvw, axis=0)
E_ANGwu_2d = spec.E_ANGwu.transpose();  E_ANGwu_med = np.nanmedian(spec.E_ANGwu, axis=0)
E_COHuv_2d = spec.E_ANGwu.transpose();  E_COHuv_med = np.nanmedian(spec.E_COHuv, axis=0)
E_COHvw_2d = spec.E_ANGwu.transpose();  E_COHvw_med = np.nanmedian(spec.E_COHvw, axis=0)
E_COHwu_2d = spec.E_ANGwu.transpose();  E_COHwu_med = np.nanmedian(spec.E_COHwu, axis=0)
E_PHAuv_2d = spec.E_ANGwu.transpose();  E_PHAuv_med = np.nanmedian(spec.E_PHAuv, axis=0)
E_PHAvw_2d = spec.E_ANGwu.transpose();  E_PHAvw_med = np.nanmedian(spec.E_PHAvw, axis=0)
E_PHAwu_2d = spec.E_ANGwu.transpose();  E_PHAwu_med = np.nanmedian(spec.E_PHAwu, axis=0)

p_max1 = p_raw_max + spec.cf/10; p_min1 = p_raw_min + spec.cf/10
p_max1 = np.ceil(np.log10(np.amax([np.amax(spec.EuEu), np.amax(spec.EvEv), np.amax(spec.EwEw)]))*5)/5

f_min0 = freq_1d[0];  f_max0 = freq_1d[N_frequency-1]
print(spec.str, "   f_MAX/min:", f_max0, f_min0, "   num of f, step, window:", N_frequency, data.freq_step[0][0], data.freq_width[0][0])

## Spec

In [ ]:
n0 = 0;  n1 = n_time0//2;   n2 = n_time0-1
f_min = f_min0;  f_max = f_max0
p_min = p_min0;  p_max = p_max0

fig = plt.figure(figsize=(14, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)
else:
    ax1 = fig.add_subplot(1, 1, 1)

ax1.plot(freq1, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq1, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq1, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq1, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, spec.EuEu[n2], '-r',  linewidth=1.0, label='uu end')
ax1.plot(freq1, spec.EvEv[n2], '-g',  linewidth=0.8, label='vv end')
ax1.plot(freq1, spec.EwEw[n2], '-b',  linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
#
if comp_mode > 0:    # Matrix
    ax2.plot(freq1, spec.E_DoLuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax2.plot(freq1, spec.E_DoLvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax2.plot(freq1, spec.E_DoLwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax2.plot(freq1, spec.E_DoLuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax2.plot(freq1, spec.E_DoLvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax2.plot(freq1, spec.E_DoLwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax2.plot(freq1, spec.E_DoLuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax2.plot(freq1, spec.E_DoLvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax2.plot(freq1, spec.E_DoLwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax3.plot(freq1, spec.E_DoCuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax3.plot(freq1, spec.E_DoCvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax3.plot(freq1, spec.E_DoCwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax3.plot(freq1, spec.E_DoCuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax3.plot(freq1, spec.E_DoCvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax3.plot(freq1, spec.E_DoCwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax3.plot(freq1, spec.E_DoCuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax3.plot(freq1, spec.E_DoCvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax3.plot(freq1, spec.E_DoCwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax4.plot(freq1, spec.E_ANGuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax4.plot(freq1, spec.E_ANGvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax4.plot(freq1, spec.E_ANGwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax4.plot(freq1, spec.E_ANGuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax4.plot(freq1, spec.E_ANGvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax4.plot(freq1, spec.E_ANGwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax4.plot(freq1, spec.E_ANGuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax4.plot(freq1, spec.E_ANGvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax4.plot(freq1, spec.E_ANGwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax5.plot(freq1, spec.E_DoLuv[n0], ':r',  linewidth=1.0, label='uv init')
    if f_mode == 1:
        ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

ax1.set_ylabel(spec.str)
if comp_mode > 0:    # Matrix
    ax4.set_xlabel('frequency [Hz]');  ax2.set_ylabel('Deg. Linear Pol.');  ax3.set_ylabel('Deg. Circular Pol.');  ax4.set_ylabel('Linear Deg.')    
else:
    ax1.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8)
ax3.legend(loc='upper right', fontsize=8); ax4.legend(loc='upper right', fontsize=8) 

xlim=[f_min, f_max];  ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    ylim=[-0.05, 1.05]; ax2.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax3.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

In [ ]:
n0 = 0;  n1 = n_time0//2;   n2 = n_time0-1
f_min = f_min0;  f_max = f_max0
p_min = p_min0;  p_max = p_max0

fig = plt.figure(figsize=(14, 11));  ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(freq1, EE_med  ,      '-y', linewidth=4.0, label='EE_med')
ax1.plot(freq1, EuEu_med,      '-r', linewidth=2.0, label='EuEu_med')
ax1.plot(freq1, EvEv_med,      '-g', linewidth=2.0, label='EvEv_med')
ax1.plot(freq1, EwEw_med,      '-b', linewidth=2.0, label='EwEw_med')
ax1.plot(freq1, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq1, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq1, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq1, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, spec.EuEu[n2], '-r',  linewidth=1.0, label='uu end')
ax1.plot(freq1, spec.EvEv[n2], '-g',  linewidth=0.8, label='vv end')
ax1.plot(freq1, spec.EwEw[n2], '-b',  linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')

ax1.set_ylabel(spec.str); ax1.set_xlabel('frequency [Hz]')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S');  title_date = "[" + data_name + "]  " + date2;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim); ax7.set_xlim(xlim); # ax3.set_xlim(xlim); ax4.set_xlim(xlim); 
ylim=[10**p_min, 10**p_max]; 
ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

## FT

In [ ]:
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
    ax4.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
    ax3.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
ax1.set_ylabel('EuEu [kHz]'); ax2.set_ylabel('EvEv [kHz]'); ax3.set_ylabel('EwEw [kHz]')
ax1.set_title("[" + data_name + "]")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min==0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        E_min = '2024-08-20 20:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2024-08-20 23:10:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
        xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

In [ ]:
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(1, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('EE [kHz]');  ax1.set_title("[" + data_name + "]  ")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min==0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        E_min = '2024-08-20 22:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2024-08-20 23:20:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
        xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT2-log.png'
    fig2d.savefig(png_fname)

## Ang-T

In [ ]:
if comp_mode > 0:    # Matrix
    f_min = f_min0;      f_max = f_max0
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max
    p_min = p_min0;      p_max = p_max0

    fig2d = plt.figure(figsize=[16,11])
    if time_mode == 1:
        ax1 = fig2d.add_subplot(10, 1, 1); ax2 = fig2d.add_subplot(10, 1, 2); ax3 = fig2d.add_subplot(10, 1, 3)
        ax4 = fig2d.add_subplot(10, 1, 4); ax5 = fig2d.add_subplot(10, 1, 5); ax6 = fig2d.add_subplot(10, 1, 6)
        ax7 = fig2d.add_subplot(10, 1, 7); ax8 = fig2d.add_subplot(10, 1, 8); ax9 = fig2d.add_subplot(10, 1, 9)
        ax10 = fig2d.add_subplot(10, 1, 10);                                                                     ax10.set_xlabel(str_date)
    else:
        ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
        ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
        ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9);  ax9.set_xlabel(str_date)

    ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max); ax5.set_ylim(f_min, f_max)
    ax6.set_ylim(f_min, f_max); ax7.set_ylim(f_min, f_max); ax8.set_ylim(f_min, f_max); ax9.set_ylim(f_min, f_max)
    if f_mode == 1:
        ax1.set_yscale('log');  ax2.set_yscale('log');      ax3.set_yscale('log');      ax4.set_yscale('log');      ax5.set_yscale('log')
        ax6.set_yscale('log');  ax7.set_yscale('log');      ax8.set_yscale('log');      ax9.set_yscale('log')
    ax1.set_ylabel('D.Lin UV'); ax2.set_ylabel('D.Lin VW'); ax3.set_ylabel('D.Lin WU')
    ax4.set_ylabel('D.Cir UV'); ax5.set_ylabel('D.Cir VW'); ax6.set_ylabel('D.Cir WU')
    ax7.set_ylabel('D.Ang UV'); ax8.set_ylabel('D.Ang VW'); ax9.set_ylabel('D.Ang WU'); ax1.set_title("[" + data_name + "]"'')

    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.');   pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical")
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical"); pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical"); pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")
    
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        if t_min==0:
            xlim=[Epoch_1d[0], Epoch_1d[-1]]
        else:
            E_min = '2024-08-22 01:59:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
            E_max = '2024-08-22 02:25:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
            xlim=[t_min, t_max]
        print("==>", xlim)
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.subplots_adjust(hspace=0.02);  plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT-log.png'
        fig2d.savefig(png_fname)

# CLEANED data

In [ ]:
# RFI rejection:  Mode -- 0:median   1:max reject   2:min
EuEu_c = juice_math.clean_rfi(np.ravel(spec.EuEu), 5, 2);  EvEv_c = juice_math.clean_rfi(np.ravel(spec.EvEv), 5, 2)
EwEw_c = juice_math.clean_rfi(np.ravel(spec.EwEw), 5, 2);  EE_c   = juice_math.clean_rfi(np.ravel(spec.EE),   5, 2)
EuEu_c = EuEu_c.reshape(n_time0, n_freq0);                 EvEv_c = EvEv_c.reshape(n_time0, n_freq0)
EwEw_c = EwEw_c.reshape(n_time0, n_freq0);                 EE_c   = EE_c.reshape  (n_time0, n_freq0)
EuEu_c_2d = EuEu_c.transpose();  EvEv_c_2d = EvEv_c.transpose();  EwEw_c_2d = EwEw_c.transpose();  EE_c_2d = EE_c.transpose()

## CLEANED - Spec

In [ ]:
n0 = 0;  n1 = n_time0//2;  n2 = n_time0-1
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
p_min = p_min0;  p_max = p_max0

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(freq1, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq1, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq1, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq1, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, spec.EuEu[n2], '-r',  linewidth=1.0, label='uu end')
ax1.plot(freq1, spec.EvEv[n2], '-g',  linewidth=0.8, label='vv end')
ax1.plot(freq1, spec.EwEw[n2], '-b',  linewidth=0.6, label='ww end')
ax2.plot(freq1, EuEu_c[n0],    ':r',  linewidth=1.0, label='uu init cleaned')
ax2.plot(freq1, EvEv_c[n0],    ':g',  linewidth=0.8, label='vv init cleaned')
ax2.plot(freq1, EwEw_c[n0],    ':b',  linewidth=0.6, label='ww init cleaned')
ax2.plot(freq1, EuEu_c[n1],    '-.r', linewidth=1.0, label='uu half cleaned')
ax2.plot(freq1, EvEv_c[n1],    '-.g', linewidth=0.8, label='vv half cleaned')
ax2.plot(freq1, EwEw_c[n1],    '-.b', linewidth=0.6, label='ww half cleaned')
ax2.plot(freq1, EuEu_c[n2],    '-r',  linewidth=1.0, label='uu end cleaned')
ax2.plot(freq1, EvEv_c[n2],    '-g',  linewidth=0.8, label='vv end cleaned')
ax2.plot(freq1, EwEw_c[n2],    '-b',  linewidth=0.6, label='ww end cleaned')
ax3.plot(freq1, spec.EE[n0],   ':r',  linewidth=1.0, label='EE init')
ax3.plot(freq1, spec.EE[n1],   ':g',  linewidth=1.0, label='EE half')
ax3.plot(freq1, spec.EE[n2],   ':b',  linewidth=1.0, label='EE end')
ax3.plot(freq1, EE_c[n0],      '-r',  linewidth=1.0, label='EE init cleaned')
ax3.plot(freq1, EE_c[n1],      '-g',  linewidth=1.0, label='EE half cleaned')
ax3.plot(freq1, EE_c[n2],      '-b',  linewidth=1.0, label='EE end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log');     ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');     ax3.set_xscale('log')
ax1.set_ylabel(spec.str);   ax2.set_ylabel(spec.str);  ax3.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title("[" + data_name + "]  " + title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max];          ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];  ax1.set_ylim(ylim); ax2.set_ylim(ylim); ax3.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
png_fname = work_dir+data_name+'_spec-c.png'
if f_mode == 1:
    png_fname = work_dir+data_name+'_spec-log-c.png'
    # fig.savefig(png_fname

In [ ]:
n0 = 0;  n1 = n_time0//2;  n2 = n_time0-1
f_min = f_min0;  f_max = f_max0
p_min = p_min0;  p_max = p_max0

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(freq1, spec.EvEv[n0]/spec.EuEu[n0], ':r', linewidth=0.8, label='V/U init')
ax1.plot(freq1, spec.EwEw[n0]/spec.EuEu[n0], ':b', linewidth=0.6, label='W/U init')
ax1.plot(freq1, spec.EvEv[n1]/spec.EuEu[n1], '-.r', linewidth=0.8, label='V/U half')
ax1.plot(freq1, spec.EwEw[n1]/spec.EuEu[n1], '-.b', linewidth=0.6, label='W/U half')
ax1.plot(freq1, spec.EvEv[n2]/spec.EuEu[n2], '-r', linewidth=0.8, label='V/U end')
ax1.plot(freq1, spec.EwEw[n2]/spec.EuEu[n2], '-b', linewidth=0.6, label='W/U end')
ax2.plot(freq1, EvEv_c[n0]/EuEu_c[n0],       ':r', linewidth=0.8, label='V/U init cleaned')
ax2.plot(freq1, EwEw_c[n0]/EuEu_c[n0],       ':b', linewidth=0.6, label='W/U init cleaned')
ax2.plot(freq1, EvEv_c[n1]/EuEu_c[n1],       '-.r', linewidth=0.8, label='V/U half cleaned')
ax2.plot(freq1, EwEw_c[n1]/EuEu_c[n1],       '-.b', linewidth=0.6, label='W/U half cleaned')
ax2.plot(freq1, EvEv_c[n2]/EuEu_c[n2],       '-r', linewidth=0.8, label='V/U end cleaned')
ax2.plot(freq1, EwEw_c[n2]/EuEu_c[n2],       '-b', linewidth=0.6, label='W/U end cleaned')
ax3.plot(freq1, spec.EE[n0],   ':r',  linewidth=1.0, label='EE init')
ax3.plot(freq1, spec.EE[n1],   ':g',  linewidth=1.0, label='EE half')
ax3.plot(freq1, spec.EE[n2],   ':b',  linewidth=1.0, label='EE end')
ax3.plot(freq1, EE_c[n0],      '-r',  linewidth=1.0, label='EE init cleaned')
ax3.plot(freq1, EE_c[n1],      '-g',  linewidth=1.0, label='EE half cleaned')
ax3.plot(freq1, EE_c[n2],      '-b',  linewidth=1.0, label='EE end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log');     ax3.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log');     ax3.set_xscale('log')
ax1.set_ylabel("ratio");    ax2.set_ylabel("ratio - cleaned");   ax3.set_ylabel(spec.str);  ax3.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title("[" + data_name + "]  " + title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max];          ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];  ax1.set_ylim(ylim); ax2.set_ylim(ylim); ax3.set_ylim(ylim)

fig.subplots_adjust(hspace=0); fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_ratio.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_ratio-log.png'
    # fig2d.savefig(png_fname)

## CLEANED -- FT

In [ ]:
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
    ax4.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
    ax3.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
ax1.set_ylabel('EuEu cleaned[kHz]'); ax2.set_ylabel('EvEv cleaned [kHz]'); ax3.set_ylabel('EwEw cleaned [kHz]')
ax1.set_title("[" + data_name + "]")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min==0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        E_min = '2024-08-20 20:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2024-08-20 23:10:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
        xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-c-log.png'
    fig2d.savefig(png_fname)

In [ ]:
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(1, 1, 1);  ax2 = fig2d.add_subplot(2, 1, 2);  ax2.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(1, 1, 1);                                     ax1.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('EE [kHz]');  ax1.set_title("[" + data_name + "]  ")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EE_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp1 = fig2d.colorbar(p1, ax=ax2, orientation="vertical"); pp1.set_label(spec.str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min==0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        E_min = '2024-08-20 22:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2024-08-20 23:20:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
        xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT2-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT2-c-log.png'
    fig2d.savefig(png_fname)